In [ ]:
! unzip /content/sample_data/final_docs.zip

Archive:  /content/sample_data/final_docs.zip
   creating: final_docs/
  inflating: final_docs/专业提升.md  
  inflating: final_docs/公司2025年安全技术劳动保护措施项目信息表.pdf_optim_notable.md  
  inflating: final_docs/内网门户使用帮助.pdf_optim_notable.md  
  inflating: final_docs/办公室（党委办公室）_岗位绩效工资管理实施方案及薪酬相关政策解读_20230716.pdf_optim_notable.md  
  inflating: final_docs/办公室（党委办公室）_技能等级评价方式一览表_20230716.pdf_optim_notable.md  
  inflating: final_docs/员工规章制度应知应会手册.pdf_optim_notable.md  
  inflating: final_docs/国网后勤服务标准及规范.pdf_optim_notable.md  
  inflating: final_docs/国网陕西省电力公司补充医疗保险实施办法.pdf_optim_notable.md  
  inflating: final_docs/国网陕西省电力有限公司2025年离退休工作要点.pdf_optim_notable.md  
  inflating: final_docs/国网陕西省电力有限公司AED管理暂行规定.pdf_optim_notable.md  
  inflating: final_docs/国网陕西省电力有限公司“宽带岗级”岗位绩效工资实施细则.pdf_optim_notable.md  
  inflating: final_docs/国网陕西省电力有限公司应执行规章制度目录（2025版）.pdf_optim_notable.md  
  inflating: final_docs/国网陕西省电力有限公司本部绩效考核管理办法.pdf_optim_notable.md  
  inflating: final_docs/培训相关要求.pdf_optim_notable.md  
  i

In [ ]:
! pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [ ]:
import os
from langchain_core.documents.base import Document as LangchainDocument
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm
import json
def load_docs(data_dir):
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"❌ 文档文件夹不存在：{data_dir}")

    md_files = []
    for root, _, files in os.walk(data_dir):
        for f in files:
            if f.endswith("_notable.md"):
                md_files.append(os.path.join(root, f))

    if not md_files:
        raise ValueError(f"❌ 无 md 文件：{data_dir}")

    # 读取文档内容
    raw_docs = []
    for path in tqdm(md_files, desc="📄 加载文档"):
        with open(path, "r", encoding="utf-8") as f:
            content = f.read()
        raw_docs.append(LangchainDocument(
            page_content=content,
            metadata={"source": os.path.basename(path).replace("_notable","").replace("_optim","").replace(".md","")}  # 记录文件名作为来源
        ))

    # 切分文档
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=512,
            chunk_overlap=50,
            add_start_index=True,
            strip_whitespace=True,
            separators=["#","![]","\n\n", "\n",".", " ", ""],
        )
    doc_processed = []
    for doc in raw_docs:
        doc_processed += text_splitter.split_documents([doc])

    print(f"✅ 共处理 {len(doc_processed)} 个文档块")
    return doc_processed


doc_processed=load_docs("/content/final_docs")

📄 加载文档: 100%|██████████| 29/29 [00:00<00:00, 4445.23it/s]

✅ 共处理 912 个文档块


In [ ]:
doc_processed[0]

Document(metadata={'source': '职称申报规范国网人才〔2025〕11号.pdf', 'start_index': 0}, page_content='普通事项\n\n# 国网人才交流服务中心有限公司文件\n\n国网人才［2025］11号\n\n# 职称申报规范\n\n（2025年5月修订）\n\n各有关单位：\n\n为准确把握各专业系列职称评定标准，正确履行职称评定工作程序，依据《国家电网有限公司职称评定管理办法》，国网人才交流服务中心有限公司（以下简称“国网人才中心”）对职称申报规范明确如下。\n\n# 一、基本规定\n\n# （一）时间规定\n\n计算现有职称取得年限、业绩成果取得时间或从事专业技术工作年限的截止时间，均为职称申报年度的12月31日。')

In [ ]:


def documents_to_jsonl(documents, output_path):
    """
    将Document对象列表转换为JSON Lines文件

    参数:
        documents: Document对象的列表，每个对象包含metadata和page_content
        output_path: 输出的.jsonl文件路径（如"output.jsonl"）
    """
    with open(output_path, "w", encoding="utf-8") as f:
        for doc in documents:
            # 提取Document中的关键信息，构建JSON对象
            json_obj = {
                # 合并metadata中的键值对（如source、start_index）
                **doc.metadata,
                # 添加page_content字段
                "context": doc.page_content
            }
            # 将JSON对象转为字符串，写入文件（每行一个）
            # ensure_ascii=False：保留中文等非ASCII字符
            f.write(json.dumps(json_obj, ensure_ascii=False) + "\n")


documents_to_jsonl(doc_processed, "output.jsonl")
print("转换完成，文件已保存至 output.jsonl")



转换完成，文件已保存至 output.jsonl


In [ ]:
# 将数据转换为huggingface dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/output.jsonl")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset["train"][0]

{'source': '职称申报规范国网人才〔2025〕11号.pdf',
 'start_index': 0,
 'context': '普通事项\n\n# 国网人才交流服务中心有限公司文件\n\n国网人才［2025］11号\n\n# 职称申报规范\n\n（2025年5月修订）\n\n各有关单位：\n\n为准确把握各专业系列职称评定标准，正确履行职称评定工作程序，依据《国家电网有限公司职称评定管理办法》，国网人才交流服务中心有限公司（以下简称“国网人才中心”）对职称申报规范明确如下。\n\n# 一、基本规定\n\n# （一）时间规定\n\n计算现有职称取得年限、业绩成果取得时间或从事专业技术工作年限的截止时间，均为职称申报年度的12月31日。'}

In [ ]:
import random
import time
from typing import List
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document  # 假设LangchainDocument是Document的别名

# 从环境变量获取API密钥
from google.colab import userdata
api_key = userdata.get('siliconflow')

# 初始化LLM客户端（ChatOpenAI支持多线程调用，线程安全）
llm_client = ChatOpenAI(
    base_url="https://api.siliconflow.cn/v1",
    model="Qwen/Qwen2.5-14B-Instruct",
    api_key=api_key,
    temperature=0.2
)

# QA生成提示词（保持不变）
QA_generation_prompt = """
你的任务是根据一段上下文，撰写一个事实型问题和对应的答案。
你所撰写的事实型问题，需能通过从上下文中提取一条具体、简洁的事实信息来回答。
该事实型问题的格式需与用户在搜索引擎中提问的风格一致——这意味着，你的事实型问题**不得提及**“根据这段文字”“上下文”这类表述。

常问问题的风格示例：
- 工会的主要职责有哪些？
- 国家电网有限公司劳模先进评选表彰管理办法的评选标准是什么
- 工伤认定的程序是怎样的？
- 工伤认定申请需要哪些具体材料
- 用人单位未及时申请工伤认定时，责任由谁承担？

请按以下格式提供答案：

Output:::
Factoid question:（你撰写的事实型问题）
Answer:（你针对该事实型问题给出的答案）

以下是上下文。

Context: {context}\n
Output:::"""


def call_llm(llm_client, prompt: str):
    """同步调用LLM，返回响应内容（保持不变）"""
    response = llm_client.invoke(prompt)
    return response.content


def process_single_doc(llm_client, doc: Document):
    """处理单个文档：生成QA并解析结果（供线程池调用的任务函数）"""
    try:
        # 生成prompt并调用LLM
        prompt = QA_generation_prompt.format(context=doc.page_content)
        output_QA_couple = call_llm(llm_client, prompt)

        # 解析问题和答案
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0].strip()
        answer = output_QA_couple.split("Answer: ")[-1].strip()

        # 过滤过长的答案
        if len(answer) >= 500:
            return None

        # 返回处理结果
        return {
            "context": doc.page_content,
            "question": question,
            "answer": answer,
            "source_doc": doc.metadata.get("source", "unknown")
        }
    except Exception as e:
        # 捕获解析或调用异常，返回None（不中断其他任务）
        print(f"处理文档失败：{str(e)}")
        return None


def generate_qa_dataset(llm_client, docs_processed: List[Document], max_workers: int = 5):
    """多线程生成QA数据集"""
    # 采样文档（保持原有逻辑）
    N_GENERATIONS = len(docs_processed)
    sampled_docs = random.sample(docs_processed, N_GENERATIONS)
    print(f"生成 {N_GENERATIONS} 组QA数据（并发数：{max_workers}）...")

    outputs = []
    # 创建线程池，max_workers控制并发数（根据API限流调整）
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务到线程池，获取future对象（跟踪任务状态）
        futures = {executor.submit(process_single_doc, llm_client, doc): doc for doc in sampled_docs}

        # 遍历完成的任务，收集结果（用tqdm显示进度）
        for future in tqdm(as_completed(futures), total=len(futures), desc="生成QA中"):
            result = future.result()
            if result is not None:  # 只保留成功解析的结果
                outputs.append(result)

    print(f"处理完成，有效QA数据：{len(outputs)}/{N_GENERATIONS}")
    return outputs


# 调用示例（假设doc_processed是Document对象列表）
QA_dataset = generate_qa_dataset(llm_client, doc_processed, max_workers=5)  # 并发数设为5（可根据API限制调整）


生成 912 组QA数据（并发数：5）...


生成QA中:  95%|█████████▌| 870/912 [05:09<00:28,  1.47it/s]

处理文档失败：Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}


生成QA中:  96%|█████████▌| 872/912 [05:09<00:17,  2.28it/s]

处理文档失败：Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}


...

处理文档失败：Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}


生成QA中:  98%|█████████▊| 893/912 [05:23<00:13,  1.46it/s]

处理文档失败：Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}处理文档失败：Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}



生成QA中: 100%|██████████| 912/912 [05:35<00:00,  2.72it/s]

处理完成，有效QA数据：907/912


In [ ]:
QA_dataset[0]

{'context': '综上所述，表格主要展示了不同岗位的数量以及它们之间的关系和特殊需求。\n\n根据提供的表格信息，我们可以看到不同级别的职员（从七级到八级）以及他们对应的某些编号或标识符。例如：\n\n- 七级职员有12个条目，并且其编号范围包括35至24。\n- 八级职员有两个条目（可能是重复输入），并且其编号范围包括31至20。\n\n表格中还包含了一些未具体定义的“一”和空白单元格，这可能表示某些特定条件或状态，但没有足够的信息来确定它们的确切含义。此外，“L”字符出现在表格中，但它的意义也不明确。\n\n由于表格中的数据较为零散且部分信息缺失（如第一列和第二列的内容），很难给出一个全面的解读。不过，从现有信息可以推测这可能是一个关于职员等级及其对应编号或标识符的记录表。',
 'question': '七级职员的编号范围是多少？',
 'answer': '七级职员的编号范围是35至24。',
 'source_doc': '陕电人资〔2023〕76号\u3000国网陕西省电力有限公司关于印发《国网陕西省电力有限公司“宽带岗级”岗位绩效工资实施细则》的通知.pdf'}

In [ ]:
#过滤掉一部分qa
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

question_groundedness_critique_prompt = """
你将获得一段上下文和一个问题。
你的任务是给出一个「总体评分」，用于评估利用给定上下文能否清晰、无歧义地回答该问题的程度。
评分范围为1到5分：1分表示根据给定上下文完全无法回答该问题，5分表示利用给定上下文能清晰、无歧义地回答该问题。

请按以下格式提供答案：

Answer:::
Evaluation:（说明给出该评分的理由，以文本形式呈现）
Total rating:（你的评分，为1到5之间的数字）

你的答案中必须包含 'Evaluation:'和 'Total rating:' 对应的内容。

以下是待评估的问题和上下文。

问题：{question}\n
上下文：{context}\n
Answer::: """



question_relevance_critique_prompt = """
你将获得一个问题。
你的任务是给出一个「总体评分」，用于评估该问题对国网公司内部员工了解工会相关规章制度的有用程度。
评分范围为1到5分：1分表示该问题完全没有用处，5分表示该问题极其有用。

请按以下格式提供答案：

Answer:::
Evaluation:（说明给出该评分的理由，以文本形式呈现）
Total rating:（你的评分，为1到5之间的数字）

你的答案中必须包含 'Evaluation:'和 'Total rating:' 对应的内容。

以下是待评估的问题。

问题：{question}\n
Answer::: """



question_standalone_critique_prompt = """
你将获得一个问题。
你的任务是给出一个「总体评分」，用于评估该问题的「独立于上下文」程度。
评分范围为1到5分：1分表示该问题需要依赖额外信息才能被理解，5分表示该问题自身含义明确（无需额外信息即可理解）。

例如，若问题中提及特定场景（如“在该上下文中”或“在该文档中”），则评分必须为1分。
问题中可包含生僻的技术名词或缩写（如Gradio、Hub、Hugging Face、Space），但仍可评为5分——只要拥有文档查阅权限的操作者能明确该问题所指内容即可。


请按以下格式提供答案：

Answer:::
Evaluation:（说明给出该评分的理由，以文本形式呈现）
Total rating:（你的评分，为1到5之间的数字）

你的答案中必须包含 'Evaluation:'和 'Total rating:' 对应的内容。

以下是待评估的问题。

问题：{question}\n
Answer::: """

critic_llm_client = ChatOpenAI(
    base_url="https://api.siliconflow.cn/v1",
    model="Qwen/Qwen2.5-32B-Instruct",
    api_key=api_key,
    temperature=0.2
)

def process_single_qa(llm_client, output: Dict) -> Dict:
    """处理单个QA对的评估：生成三项指标并解析结果（供线程池调用）"""
    try:
        # 并行调用三次LLM（单个QA对的三项评估）
        evaluations = {
            "groundedness": call_llm(
                llm_client,
                question_groundedness_critique_prompt.format(
                    context=output["context"],
                    question=output["question"]
                ),
            ),
            "relevance": call_llm(
                llm_client,
                question_relevance_critique_prompt.format(
                    question=output["question"]
                ),
            ),
            "standalone": call_llm(
                llm_client,
                question_standalone_critique_prompt.format(
                    question=output["question"]
                ),
            ),
        }

        # 解析评估结果并更新到output
        for criterion, evaluation in evaluations.items():
            # Extract the numerical score by finding the last digit(s)
            try:
                score_str = evaluation.split("Total rating: ")[-1].strip()
                score = int(''.join(filter(str.isdigit, score_str)))
            except ValueError:
                score = 1 # Default to 1 if parsing fails

            eval_text = evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1].strip() # Keep the evaluation text

            # Update output fields
            output[f"{criterion}_score"] = score
            output[f"{criterion}_eval"] = eval_text

        return output  # 返回更新后的QA对

    except Exception as e:
        # 捕获解析或调用异常，返回原output（不中断其他任务）
        print(f"处理文档失败：{str(e)}") # Changed error message to be more accurate
        return output


def critic_qa(
    llm_client,
    qa_list: List[Dict],
    max_workers: int = 5  # 并发数，根据API限流调整
) -> List[Dict]:
    """多线程评估QA列表中的每个QA对"""
    print(f"对 {len(qa_list)} 个QA对进行评估（并发数：{max_workers}）...")

    # 用于存储最终结果（保持原列表顺序）
    results = [None] * len(qa_list)

    # Create a list of tasks to submit
    tasks = [(llm_client, qa) for qa in qa_list]

    # Create a dictionary to map futures back to their original index
    future_to_index = {}

    # 创建线程池，提交所有QA评估任务
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks and store the future with its original index
        for idx, (llm_client_arg, qa_arg) in enumerate(tasks):
            future = executor.submit(process_single_qa, llm_client_arg, qa_arg)
            future_to_index[future] = idx

        # Iterate through completed tasks and store results in the correct order
        for future in tqdm(as_completed(future_to_index), total=len(future_to_index), desc="评估QA中"):
            idx = future_to_index[future]
            try:
                result = future.result()
                results[idx] = result
            except Exception as exc:
                print(f'Generated an exception: {exc}') # Print exception for debugging
                results[idx] = None # Store None for failed tasks

    print(f"评估完成，共处理 {len(results)} 个QA对")
    return results


# 调用示例（假设doc_processed是Document对象列表）
results = critic_qa(critic_llm_client,QA_dataset)

对 907 个QA对进行评估（并发数：5）...


评估QA中: 100%|██████████| 907/907 [22:10<00:00,  1.47s/it]

评估完成，共处理 907 个QA对


In [ ]:
results[200]

{'context': '# 第五章专业与能力考试\n\n第二十四条参考人选的确定。加权总积分达标者方可进入专业与能力考试阶段。根据各系列中级《评定标准》和《业绩积分标准》，对申报人员申报材料进行审查、鉴定、评价、复审、公示后，加权总积分达标者，进入专业与能力考试阶段。\n\n第二十五条专业与能力考试内容。专业与能力考试内容将依据相应系列《评定标准》中“专业理论水平要求”和“工作经历和能力要求”且通过业绩积分不好界定的、且必须掌握的综合知识与综合能力等原则进行确定；按知识类和能力类2部分且各占一定比重设计考卷。知识类主要考查与申报专业相关的通用基础理论知识、专业知识，以及电力与能源战略、企业文化、相关管理规章制度等；能力类主要考查从事相应专业技术工作所应具备的综合能力。\n\n第二十六条考试时间与考题类型。专业与能力考试时间为150分钟。参考人员在考前自行打印准考证。参加考试时必须携带本人身份证和准考证到指定考场参加考试。考题类型全部为客观题，由单选、多选和判断等题型组成，满分100分。',
 'question': '专业与能力考试的时间是多少？',
 'answer': '专业与能力考试的时间为150分钟。',
 'source_doc': '职称申报规范国网人才〔2025〕11号.pdf',
 'groundedness_score': 5,
 'groundedness_eval': '上文第二十六条明确指出了专业与能力考试的时间为150分钟，这直接回答了问题中关于考试时间的具体询问。因此，根据给定的上下文，可以清晰、无歧义地回答问题。',
 'relevance_score': 2,
 'relevance_eval': '该问题询问的是专业与能力考试的时间，这与了解工会相关规章制度的直接关联性较低。工会相关规章制度通常涉及员工权益保护、福利政策、活动组织等方面的内容，而专业与能力考试的时间安排更可能属于人力资源部门的培训与发展范畴。因此，这个问题对于国网公司内部员工了解工会相关规章制度的有用程度较低。',
 'standalone_score': 2,
 'standalone_eval': '该问题询问的是“专业与能力考试”的时间长度，但没有提供具体的考试名称或上下文信息，这使得问题的含义不够明确。不同的考试可能有不同的时长，因此，除非对特定的考试有预

In [ ]:
import json

def results_to_jsonl(results, output_path):
    """
    将results列表（元素为字典）转换为JSON Lines格式文件
    :param results: 输入的结果列表（如包含200个QA条目的列表）
    :param output_path: 输出的.jsonl文件路径（如"results.jsonl"）
    """
    with open(output_path, "w", encoding="utf-8") as f:
        for item in results:
            # 将每个字典转为JSON字符串，确保中文不转义
            json_str = json.dumps(item, ensure_ascii=False)
            # 每行写入一个JSON对象
            f.write(json_str + "\n")

# 调用示例（假设results是你的数据列表）
results_to_jsonl(results, "full_dataset.jsonl")
print("转换完成，文件已保存为 full_dataset.jsonl")

转换完成，文件已保存为 full_dataset.jsonl


In [ ]:
# 统计 relevance_score > 2 的数量
count = 0
for item in results:
    # 安全获取 relevance_score（若不存在则默认0）
    score = item.get("relevance_score", 0)
    standalone_score = item.get('standalone_score',0)

    if score >=2 and standalone_score>=3:
        count += 1

print(f"relevance_score 大于2的数量为: {count}")

relevance_score 大于2的数量为: 652


In [ ]:
#过滤数据集
import pandas as pd
def filter_qa(generated_questions):
    print("Filtering QA couples...")
    generated_questions = pd.DataFrame(generated_questions)
    generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 2)
    & (generated_questions["standalone_score"] >= 3)
]

    return generated_questions

final_datasets = filter_qa(results)

Filtering QA couples...


In [ ]:
print(len(final_datasets))

572


In [ ]:
final_datasets.head()

,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval
1,# （二）其他直属单位兼职培训员数量要求\n\n1.各单位市公司、下属分公司的每个部门各不少...,各单位市公司、下属分公司的每个部门至少需要多少名兼职培训员？,各单位市公司、下属分公司的每个部门至少需要1名兼职培训员。,培训相关要求.pdf,5,上文明确指出“各单位市公司、下属分公司的每个部门各不少于1人（1-2人）”，这直接回答了问题...,3,该问题涉及国网公司内部的人员配置情况，特别是关于兼职培训员的数量要求，这与工会相关规章制度有...,4,该问题虽然指定了具体的组织结构（各单位市公司、下属分公司、每个部门），但并未提及特定的上下文...
2,# 第五章 正高级职称评定标准\n\n# 第十三条 基本要求\n\n（一）取得相应学历、学位...,正高级职称评定的基本要求有哪些？,取得相应学历、学位证书；取得主任记者或主任编辑职称证书，并满足相应专业工作年限；取得新闻记者证。,职称申报规范国网人才〔2025〕11号.pdf,5,上文提供了正高级职称评定的基本要求，包括学历学位证书、主任记者或主任编辑职称证书及相应的工作...,3,此问题关于正高级职称评定的基本要求，对于国网公司内部员工来说，了解职称评定的相关规定有助于他...,4,该问题询问的是正高级职称评定的基本要求，虽然涉及到特定的职业评定领域，但问题本身已经提供了足...
3,# 1、个人所得税政策\n\n个人所得税以所得人为纳税人，以支付所得单位或者个人为扣缴义务人...,当全年的应纳税所得额超过660000元但不超过960000元时，个人所得税的适用税率是多少？,当全年的应纳税所得额超过660000元但不超过960000元时，个人所得税的适用税率是35%。,办公室（党委办公室）_岗位绩效工资管理实施方案及薪酬相关政策解读_20230716.pdf,5,根据给定的上下文，可以清晰、无歧义地找到当全年的应纳税所得额超过660000元但不超过960...,2,该问题主要涉及个人所得税的税率计算，与国网公司内部员工了解工会相关规章制度的直接关联性较低。...,5,该问题明确提出了一个具体的税务计算场景，即当个人的应纳税所得额落在660000元至96000...
4,# 《国家电网有限公司安全教育培训工作规定》\n\n国网（安监/4）984-2019\n\n...,新入职人员的安全教育培训内容包括哪些方面？,新入职人员的安全教育培训内容主要包括：本单位安全生产情况；安全基本知识；安全生产规章制度、安...,员工规章制度应知应会手册.pdf,5,上文详细列出了新入职人员安全教育培训的内容，包括安全生产情况、安全基本知识、安全生产规章制度...,2,该问题主要关注的是新入职人员的安全教育培训内容，这与工会相关规章制度的直接关联性较低。工会规...,4,该问题询问的是新入职人员的安全教育培训内容，虽然它涉及特定的职业培训场景，但并不依赖于具体的...
8,# 13.1一般注意事项\n\n13.1.1任何人进入生产现场（办公室、控制室、值班室和检修...,在检修期间需将栏杆拆除时，应采取什么措施？,应装设临时遮栏，并在检修结束时将栏杆立即装回。,电力安全工作规程_安全规程电缆部分.pdf,5,根据给定的上下文，可以清晰地了解到在检修期间需将栏杆拆除时，应采取的措施是装设临时遮栏，并在...,2,该问题主要涉及的是安全措施和操作规程，而不是直接与工会相关规章制度有关。虽然了解此类信息对于...,4,该问题描述了一个具体的场景，即“在检修期间需将栏杆拆除时”，并询问在这种情况下应采取什么措施...


In [ ]:
import pandas as pd
import json

def df_to_jsonl(df, output_path):
    """
    将 Pandas DataFrame 转换为 JSON Lines 格式文件。
    :param df: 要转换的 DataFrame。
    :param output_path: 输出的 .jsonl 文件路径。
    """
    with open(output_path, "w", encoding="utf-8") as f:
        # 逐行遍历 DataFrame
        for _, row in df.iterrows():
            # 将每行转换为字典，并写入 JSON 行（确保中文不转义）
            json.dump(row.to_dict(), f, ensure_ascii=False)
            f.write("\n")  # 每行一个 JSON 对象，用换行分隔



# 转换为 JSON Lines
df_to_jsonl(final_datasets, "final_datasets.jsonl")

print("已将过滤后的数据集保存为 final_datasets.jsonl")

已将过滤后的数据集保存为 final_datasets.jsonl
